In [1]:
import pandas as pd
#pd.set_option('future.no_silent_downcasting', True)

In [7]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'C:\\Documents\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [8]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [9]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [10]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2025-06-05,12.279029,883.389453,815.18
1,1,2025-06-10,NaN,NaN,1958.91
2,1,2025-06-13,NaN,NaN,774.88
3,1,2025-06-15,NaN,NaN,574.18
4,1,2025-06-16,NaN,NaN,339.42
...,...,...,...,...,...
3647,(en blanco),2025-06-28,4.980080,NaN,NaN
3648,(en blanco),2025-07-10,3.136285,NaN,NaN
3649,(en blanco),2025-07-13,2.374592,NaN,NaN
3650,(en blanco),2025-07-15,1.180094,NaN,NaN


In [11]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador

    df_param['ENTREGAS'] = suma_acumulador
    
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [12]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [13]:
list_df = []

In [14]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

In [15]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [16]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2025-06-05,12.279029,883.389453,815.18
1,1,2025-06-18,65.491810,3766.706001,4297.47
2,1,2025-06-28,49.087510,3394.102345,3824.55
3,1,2025-07-02,18.103557,1222.139884,1095.67
4,1,2025-07-20,4.239158,256.932308,814.86
...,...,...,...,...,...
1903,625,2025-07-05,4.203167,220.092130,0.00
1904,625,2025-07-10,4.400645,230.432717,0.00
1905,625,2025-07-15,3.639267,190.564380,0.00
1906,625,2025-07-20,2.029491,106.271060,0.00


In [18]:
df_combinado.to_excel(r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')